In [2]:
#pip install --upgrade llama_index llama-index-llms-azure-openai llama-index-graph-stores-neo4j llama-index-embeddings-azure-openai

In [33]:
from utilitiesPOC import *

from auths import openaisecret, embeddings_secret, adls_secret

storage_account =  '{"storage_account_key": "","storage_container": "manufacturing-copilot","storage_account": "euwdsrg03rsta07dls01"}'
path = "Documents/Embeddings/"



llmconfig= '{"mode": "langchain", "auth": "token", "model": "gpt-35-turbo-16k", "deployment_name": "mxdrca","api_version": "2023-05-15","api_key_secret":"","api_endpoint": "https://usedoai0efaoa03.openai.azure.com/"}'
embedconfig= '{"embedmode": "langchain","model":"text-embedding-ada-002","deployment_name":"azure_embedding","api_key_secret":"","embeddingurl":"https://usedoai0efaoa03.openai.azure.com/"}'


storage_account = json.loads(storage_account)
storage_account_key, container, storage_account = storage_account.get('storage_account_key'),storage_account.get('storage_container'),storage_account.get('storage_account')

#read from file
storage_account_key = adls_secret

llmconfig = json.loads(llmconfig)
embedconfig = json.loads(embedconfig)

embedmode, embedmodel,embed_deployment,embed_key, embed_url = embedconfig.values()
mode, auth, model, deployment_name, api_version, api_key_secret, api_endpoint = llmconfig.values()

api_key_secret = openaisecret
embed_key = embeddings_secret

# COMMAND ----------

#GETTING THE SECRETS
api_key = api_key_secret


embed_key = embed_key

In [34]:

# DBTITLE 1,Define LLM 
#DEFINE LLM USING AZURE OPEN AI
llmrca = getllm(openai=True,azure =True,mode = mode, auth=auth,model=model,dep_name = deployment_name,api_version= api_version,
                     api_key =api_key,api_base = api_endpoint)


embeddings_model  = getembeddings(model=embedmodel,dep_name=embed_deployment,api_key=embed_key,api_base=embed_url)

# COMMAND ----------

mount_point = '/mnt/copilotembedds'


#@async_retry
def read_embeddings(fromadls=False,dbricks=False,**kwargs):
    if fromadls:
        if dbricks:
            #reads it into a dataframe first from the mount point
            storage_account=storage_account,container=container,mount_point=mount_point,path=path,storage_account_key=storage_account_key
            for mount in dbutils.fs.mounts():
                if mount.mountPoint.startswith(mount_point):
                    print("Unmounting mnt point")
                    dbutils.fs.unmount(mount.mountPoint)
            
            #mount to path
            dbutils.fs.refreshMounts() #refresh mounts to avoid errors

            mntfiles = mount_to_local(storage_account, container, mount_point,path, storage_account_key)
            #reads into dataframe
            adlsembedspath = [e.path for e in dbutils.fs.ls(mntfiles)][-1] #read last file 
            adlsembedspath= adlsembedspath.replace("dbfs:","/dbfs")
            df = pd.read_pickle(adlsembedspath)

        else:
            #read from adls but not from databricks
            from azure.storage.filedatalake import DataLakeServiceClient
            storage_account,container,path,storage_account_key=kwargs.get("storage_account"),kwargs.get("container"),kwargs.get("path"),kwargs.get("storage_account_key")
            connstr = f"DefaultEndpointsProtocol=https;AccountName={storage_account};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
            service = DataLakeServiceClient.from_connection_string(conn_str=connstr)
            file_system_client = service.get_file_system_client(container)
            paths = file_system_client.get_paths(path=path)
            files = [p for p in paths if not p.is_directory]
            files.sort(key=lambda x: x.last_modified)
            last_file = files[-1].name
            file_client = file_system_client.get_file_client(last_file)
            
            # Download the file content and load it as a Python object
            download = file_client.download_file()
            downloaded_bytes = download.readall()
            df = pickle.loads(downloaded_bytes)
            #df = pd.read_pickle(data)
            
    else:
        #read from local FS
        adlsembedspath = [n for n in os.listdir(path)][-1]
        
        df = pd.read_pickle(adlsembedspath)
    #returns the texts and embeddings
    txtlist = [t for t in df.texts]
    embeddings = [e for e in df.embeddings]
    return txtlist,embeddings





def read_embeddings_fromfs(path):
    #service = DataLakeServiceClient.from_connection_string(conn_str="my_connection_string")

    adlsembedspath = [n for n in os.listdir(path)][-1]        
    df = pd.read_pickle(path+"/"+adlsembedspath)
    #returns the texts and embeddings
    txtlist = [t for t in df.texts]
    embeddings = [e for e in df.embeddings]
    return txtlist,embeddings




# COMMAND ----------


loaded_embeddings = read_embeddings(fromadls=True,dbricks=False,storage_account=storage_account,container=container,path=path,storage_account_key=storage_account_key)#read_embeddings_fromfs(path)

# COMMAND ----------

from dataclasses import dataclass
from typing import ClassVar
import pickle

@dataclass 
class RAG2:
    memory_conversation  = None
    def __init__(self,llm,graph,type="langchain",embeddings=None,usememory=False):
        self.llm = llm
        self.graph = graph
        self.type = type
        self.embeddings = embeddings
        self.usememory = usememory

    @staticmethod
    def memory(type='langchain',memory_key="chat_history",output_key='result',rmessages=True,readmem=None):
        if readmem:
            if type.lower()=="langchain":
                from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
                memory = ConversationBufferMemory(
                memory_key=memory_key, return_messages=rmessages)
                #output_key=output_key)
                
                readonlymemory = ReadOnlySharedMemory(memory=memory)

                return memory
        else: 
            return None
    

    def vector_retriever(self,store="chroma",**kwargs):
        texts = kwargs.get("texts")
        if store.lower()=="chroma":
            if not self.embeddings:
                self.embeddings = getembeddings(model=kwargs.get("model"),dep_name = kwargs.get("dep_name"),api_key = kwargs.get("api_key"),api_base= kwargs.get("api_base"))
            

            from langchain.chains import RetrievalQA
            persist_directory = kwargs.get("persist_directory","filesdb")
            search_kwargs = kwargs.get("search_kwargs",{"k": 3})
            from langchain.vectorstores import Chroma
            
            db = Chroma.from_documents(texts,self.embeddings,persist_directory= persist_directory)
            vec_retriever = db.as_retriever(search_kwargs=search_kwargs)
         
            return vec_retriever  

            
        elif store.lower()=="faiss":
            #to be implemented
            from langchain_community.vectorstores import FAISS
            from langchain_core.output_parsers import StrOutputParser
            from langchain_core.prompts import ChatPromptTemplate
            from langchain_core.runnables import RunnablePassthrough
            from langchain_openai import ChatOpenAI, OpenAIEmbeddings
            if not self.embeddings:
                self.embeddings = getembeddings(model=kwargs.get("model"),dep_name = kwargs.get("dep_name"),api_key = kwargs.get("api_key"),api_base= kwargs.get("api_base"))
            
            if kwargs.get("text_embeddings"):
                text_embeddings =kwargs.get("text_embeddings")
                vectorstore = FAISS.from_embeddings(text_embeddings=list(zip(text_embeddings[0],text_embeddings[1])), embedding = self.embeddings)
                retriever = vectorstore.as_retriever()
                return retriever
            
            else:
            
                vectorstore = FAISS.from_texts(texts, embedding=OpenAIEmbeddings())
                retriever = vectorstore.as_retriever()
                template = """Answer the question based only on the following context:{context} 
                Question: {query}"""
                prompt = ChatPromptTemplate.from_template(template)
                retrieval_chain = ({"context": retriever, "query": RunnablePassthrough()} | prompt
                                    | self.llm| StrOutputParser())
                return retrieval_chain
        
    def build_chain(self,**kwargs):
        #memory = self.memory() if (readmem := self.memory()) else None
        method = kwargs.get("method","graph")
        
        #PROMPT TEMPLATE        
        from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate,PromptTemplate
        #####

        memory = self.usememory
        verbose = kwargs.get("verbose",True)

        if memory:
            prompt = PromptTemplate(input_variables=["chat_history", "question"], template="{chat_history} {question}")
        else:
            prompt = PromptTemplate(input_variables=["question"],template ="{question}")

        self.memory_conversation = self.memory(readmem=memory) if not self.memory_conversation else self.memory_conversation

        if kwargs.get("falkor"):
            from langchain.chains import FalkorDBQAChain
            chain = FalkorDBQAChain.from_llm(llm=self.llm, graph=self.graph, verbose=verbose, memory=self.memory_conversation, **kwargs)
        else:
            if method.lower()=="graph":
                #general chain for graphs
                print("building graph chain")
                from langchain.chains import GraphCypherQAChain
                #Prompt template
                        
                general_system_template = r""" 
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                {context}
                ----
                """
                general_user_template = "Question:```{question}```"
                messages = [
                            SystemMessagePromptTemplate.from_template(general_system_template),
                            HumanMessagePromptTemplate.from_template(general_user_template)
                ]
                qa_prompt = ChatPromptTemplate.from_messages( messages )

                chain = GraphCypherQAChain.from_llm(llm=self.llm, prompt=prompt,graph=self.graph,validate_cypher = True, 
                                                    verbose=verbose,memory=self.memory_conversation, promptTemplate=qa_prompt,**kwargs)
                chain.input_key = "question"

            elif method.lower()=="vector":
                print("Building vector chain ")
                retriever = kwargs.get("vec_retriever")
                from langchain.chains import RetrievalQA
                from langchain.chains import ConversationalRetrievalChain
                
                # template = ("Combine the chat history and follow up question into "
                # "a standalone question. Chat History: {chat_history}"
                # "Follow up question: {query} ")
                template = ("Every time you respond you should start with This is Copilot your AI assistant")
                prompt = PromptTemplate.from_template(template)
                

                #chain = RetrievalQA.from_chain_type(llm=self.llm, chain_type="stuff", retriever=retriever,return_source_documents=True,verbose=verbose)
                        
                general_system_template = r""" 
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                {context}
                ----
                """
                general_user_template = "Question:```{question}```"
                messages = [
                            SystemMessagePromptTemplate.from_template(general_system_template),
                            HumanMessagePromptTemplate.from_template(general_user_template)
                ]
                qa_prompt = ChatPromptTemplate.from_messages( messages )
                
                chain = ConversationalRetrievalChain.from_llm(self.llm,retriever,memory=self.memory_conversation,get_chat_history=lambda h :h,
                                                              verbose=verbose,
                                                              combine_docs_chain_kwargs={"prompt": qa_prompt}) 
                
                #chain = RetrievalQA.from_chain_type(llm=self.llm, chain_type="stuff", retriever=retriever,
                #                                    return_source_documents=True,verbose=verbose,memory=self.memory_conversation,prompt=prompt)

        return chain

    def get_agent_executor(self,**kwargs):
        #combines vector and graph retrievers
        vector = self.get_retriever(method="graph")
        pass


    def get_retriever(self,**kwargs):
        method = kwargs.get("method","graph")
        complexity = kwargs.get("complexity","simple")
        
        if method.lower()=='graph':
            if self.type.lower()=="langchain":
                from langchain.chains import GraphCypherQAChain
                if complexity.lower()=='simple':
                    print("SIMPLE RAG LANGCHAIN")
                    if usememory := kwargs.get("usememory") or self.memory_conversation:
                        print("With Read memory")
                    
                    chain = self.build_chain(**kwargs)
                    return chain
                
                    
            elif self.type.lower()=="llama":
                if complexity.lower()=='simple':
                    nl2graph = kwargs.get("nl2graph",False)
                    print(nl2graph)
                    from llama_index.query_engine import RetrieverQueryEngine
                    from llama_index.retrievers import KnowledgeGraphRAGRetriever                    
                    graph_rag_retriever = KnowledgeGraphRAGRetriever(        
                    storage_context=kwargs.get("storage_context"),
                    service_context=kwargs.get("service_context"),
                    llm=self.llm,
                    verbose=True,
                    with_nl2graphquery=nl2graph)
                    query_engine = RetrieverQueryEngine.from_args(graph_rag_retriever, service_context=kwargs.get("service_context"))
                    return query_engine

                
                elif complexity.lower()=="custom":
                    if self.type.lower()=="llama":
                        from llama_index import get_response_synthesizer
                        from llama_index.query_engine import RetrieverQueryEngine
                        # create custom retriever
                        vector_retriever = VectorIndexRetriever(index=vector_index)
                        kg_retriever = KGTableRetriever(index=kg_index, retriever_mode="keyword", include_text=False)
                        custom_retriever = CustomRetriever(vector_retriever, kg_retriever)
                        # create response synthesizer
                        response_synthesizer = get_response_synthesizer(service_context=service_context,response_mode="tree_summarize")
                        custom_query_engine = RetrieverQueryEngine(retriever=custom_retriever,response_synthesizer=response_synthesizer)
                        return custom_query_engine
                    elif self.type.lower()=='langchain':
                        from langchain.agents import initialize_agent, Tool
                        #combine vector and graph retrievers
                        simple = self.get_retriever("simple",**kwargs)
                        
                        tools = [
                            Tool(name= "GraphRAG",
                                 func= simple.run,
                                 description= "Simple RAG from knowledge graph"),
                            #add more RAG tools
                        ]
        elif method.lower()=="vector":
            print("Getting vector retriever")
            vec_retriever = self.vector_retriever(**kwargs)

            chain = self.build_chain(vec_retriever=vec_retriever,**kwargs)
            return chain



    def response(self,query,complexity="simple",**kwargs):
        method = kwargs.get("method","vector")
        verbose = kwargs.get("verbose",True)
        retriever = self.get_retriever(**kwargs)
        try:
            if method.lower()=="vector":
                print("I AM USING VECTOR LETS GO")
                result =  retriever({"question": query}).get('answer')
                
            elif method.lower()=="graph":
                if self.type.lower()=='langchain':
                    print('using retriever({"query": query}).get("answer")')
                    #result = retriever({"question": query}).get('answer')#
                    result = retriever.invoke({"question": query}).get('result') 
                    pickled_str = pickle.dumps(self.memory_conversation)
                elif self.type.lower()=='llama':
                    result = retriever.query(query)
                    if 'Empty Response' in result.response:
                        raise("Error empty result or no information found")
                    else:
                        return result.response
            
            if not result or "I'm sorry, but I don't have" in result:
                print('Not using RAG')
                raise ("Error empty result or no information found in the KG")
            elif "I'm sorry, but I don't have" in result:
                raise("ERROR 404")
            else:
                return result

        except Exception as e:
            print(e)
            if self.type.lower()=='langchain':
                from langchain.schema import HumanMessage
                #message = HumanMessage(content=query)
                from langchain.chains import ConversationChain
                from langchain.prompts.prompt import PromptTemplate
                if self.usememory:
                    print("Normal LLM response with same memory")
                    prompt = PromptTemplate(input_variables=["chat_history", "question"], template="{chat_history} {query}")
                    chat = ConversationChain(llm=self.llm,verbose=kwargs.get("verbose",True),prompt=prompt,memory=self.memory_conversation ,input_key="question")
                    return chat.invoke({"question":query}).get("response")
                else:
                    from langchain.schema import HumanMessage
                    message = HumanMessage(content=query)
                    return self.llm.invoke([message])

            else:
                #return response with llama index self.llm(response)
                pass
            

# COMMAND ----------
#start graph
url ="neo4j+s://b75e0bd8.databases.neo4j.io"
username ="neo4j"
password = "aziMmumvuVru-rgeKuO0zT7mEtTxu29Jgrmp6lFKv0w"
graph_store = get_graphdb("langchain",graphdb = "neo4j",url=url,username=username,password=password)


#call RAG class initialize vector class 
rag2 = RAG2
vecrag = rag2(llmrca,graph_store,type="langchain",embeddings = embeddings_model,usememory=True)
#vectorchain = vecrag.get_retriever(method="vector",text_embeddings = loaded_embeddings,store="faiss")


# COMMAND ----------
if __name__=="__main__":
    

    # print(vectorchain({"question": "where and why did the production stoppage occurred"}).get("answer")) #if use memory we call it like this 
    # print(vectorchain({"question": "what was my previous question?"}).get("answer"))
    # #
    vecrag = rag2(llmrca,graph_store,type="langchain",embeddings = embeddings_model,usememory=True)
    print(vecrag.response(query="where and why did the production stoppage occurred",method="vector",store="faiss",text_embeddings = loaded_embeddings))
    print(vecrag.response(query="what was my previous question?",method="vector",store="faiss",text_embeddings = loaded_embeddings))
# COMMAND ----------



2024-02-22 09:00:14 [INFO]: Request URL: 'https://euwdsrg03rsta07dls01.dfs.core.windows.net/manufacturing-copilot?resource=REDACTED&directory=REDACTED&recursive=REDACTED'
Request method: 'GET'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/json'
    'User-Agent': 'azsdk-python-storage-dfs/12.14.0 Python/3.11.0 (Windows-10-10.0.22621-SP0)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '76edfe7a-d19b-11ee-a613-141333031892'
    'Authorization': 'REDACTED'
No body was attached to the request


Note: to use the LLM use it as msg = HumanMessage(content= your query) 
 llm(messages = [msg]) 
Using Azure OpenAI
Using Azure OpenAI


2024-02-22 09:00:14 [INFO]: Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json;charset=utf-8'
    'Server': 'Windows-Azure-HDFS/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '1bf0cb16-d01f-0004-07a8-65bc0e000000'
    'x-ms-version': 'REDACTED'
    'x-ms-client-request-id': '76edfe7a-d19b-11ee-a613-141333031892'
    'Date': 'Thu, 22 Feb 2024 16:00:13 GMT'
2024-02-22 09:00:14 [INFO]: Request URL: 'https://euwdsrg03rsta07dls01.blob.core.windows.net/manufacturing-copilot/Documents/Embeddings/embeddings_12022024.pkl'
Request method: 'GET'
Request headers:
    'x-ms-range': 'REDACTED'
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-dfs/12.14.0 Python/3.11.0 (Windows-10-10.0.22621-SP0)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '776c09a7-d19b-11ee-a582-141333031892'
    'Authorization': 'REDACTED'
No body was attached to the request
2024-02-22 09:00:15 [INFO]

Using langchain graph class
Getting vector retriever
Building vector chain 
I AM USING VECTOR LETS GO


2024-02-22 09:00:17 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/azure_embedding/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"


...


> Finished chain.

> Finished chain.
Based on the provided incident tickets, the production stoppage occurred on Assembly Line 3. The root cause of the stoppage can be attributed to a sudden and unplanned event. To determine the exact reason for the stoppage, a detailed analysis using Root Cause Analysis (RCA) and Failure Mode and Effects Analysis (FMEA) would be required. However, some potential factors that could have contributed to the stoppage include issues with motor surface temperature, load on the motor, voltage stability, power wiring resistance, and dust accumulation. Further investigation and analysis are necessary to identify the specific cause of the production stoppage.
Getting vector retriever
Building vector chain 
I AM USING VECTOR LETS GO


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessa

2024-02-22 09:00:20 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-22 09:00:20 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/azure_embedding/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                ﻿Incident Ticket ID: INC0007654
Issue Description: Production Line 2 experienced a sudden disruption during the morning shift due to unexpected shutdown of Benchmill impacting the production schedule.
Parameter Values: Airflow RAirflow Rate : 70 CFM, Ambient Temperature : 35 degree celsius, Motor surface temperature : 80 degree celsius, Motor rated power : 1.5 HP,
Load on motor :

2024-02-22 09:00:21 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
I apologize, but I am an AI assistant and do not have the capability to remember previous questions. Is there anything else I can assist you with?


In [35]:
rag2 = RAG2
vecrag = rag2(llmrca,graph_store,type="langchain",embeddings = embeddings_model,usememory=True)

In [36]:
print(vecrag.response(query="where and why did the production stoppage occurred",method="vector",store="faiss",text_embeddings = loaded_embeddings))
print(vecrag.response(query="what was my previous question?",method="vector",store="faiss",text_embeddings = loaded_embeddings))

2024-02-22 09:00:21 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/azure_embedding/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"


Getting vector retriever
Building vector chain 
I AM USING VECTOR LETS GO


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                ﻿Incident Ticket ID: INC0003557
Issue Description: A Sudden and unplanned production stoppage occurred on Assembly Line 3 impacting the deliverables
Parameter Values: Airflow Rate : 70 CFM, Ambient Temperature : 22 degree celsius, Motor surface temperature : 50 degree celsius, Motor rated power : 1.5 HP,
Load on motor :  70% 

2024-02-22 09:00:23 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
Based on the provided incident tickets, the production stoppage occurred on Assembly Line 3. The root cause of the stoppage can be attributed to a sudden and unplanned event. To determine the exact reason for the stoppage, a detailed analysis using Root Cause Analysis (RCA) and Failure Mode and Effects Analysis (FMEA) would be required. However, some potential factors that could have contributed to the stoppage include issues with motor performance, such as high motor surface temperature and load on the motor, as well as potential voltage stability and power wiring resistance issues. Additionally, factors such as dust accumulation and lubrication frequency could also impact the overall performance of the assembly line. A thorough investigation using RCA and FMEA methodologies would be necessary to identify the specific cause of the production stoppage.
Getting vector retriever
Building vector chain 
I AM USING VECTOR LETS GO


> Entering new LLMCha

2024-02-22 09:00:23 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-22 09:00:23 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/azure_embedding/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                ﻿Incident Ticket ID: INC0007654
Issue Description: Production Line 2 experienced a sudden disruption during the morning shift due to unexpected shutdown of Benchmill impacting the production schedule.
Parameter Values: Airflow RAirflow Rate : 70 CFM, Ambient Temperature : 35 degree celsius, Motor surface temperature : 80 degree celsius, Motor rated power : 1.5 HP,
Load on motor :

2024-02-22 09:00:24 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
I apologize, but I am an AI assistant and do not have the capability to remember previous questions. Is there anything else I can assist you with?


In [37]:
#vector
vectorchain= vecrag.get_retriever(method="vector",store="faiss",text_embeddings = loaded_embeddings)
## graoh CHAIN 
graphchain= vecrag.get_retriever(method="graph")


2024-02-22 09:00:24 [ERROR]: Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
2024-02-22 09:00:24 [ERROR]: Failed to write data to connection IPv4Address(('b75e0bd8.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))


Getting vector retriever
Building vector chain 
SIMPLE RAG LANGCHAIN
With Read memory
building graph chain


Testing them

In [38]:
vectorchain.invoke({"question":"How many tickets are there?"}).get("answer")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='where and why did the production stoppage occurred'), AIMessage(content='Based on the provided incident tickets, the production stoppage occurred on Assembly Line 3. The root cause of the stoppage can be attributed to a sudden and unplanned event. To determine the exact reason for the stoppage, a detailed analysis using Root Cause Analysis (RCA) and Failure Mode and Effects Analysis (FMEA) would be required. However, some potential factors that could have contributed to the stoppage include issues with motor performance, such as high motor surface temperature and load on the motor, as well as potential voltage stability and power wiring resistance issues. Additionally, factors such as dust accumulation and lubrication frequency could also impact 

2024-02-22 09:00:24 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-22 09:00:24 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/azure_embedding/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                ﻿Incident Ticket ID: INC0001198
Issue Description: Variations in the quality and accuracy of produced machined parts on Assembly Line 7
Parameter Values: Airflow Rate : 70 CFM, Ambient Temperature : 22 degree celsius, Motor surface temperature : 50 degree celsius, Motor rated power : 1.5 HP,
Load on motor :  70% , User workload : 60%, Voltage stability : 5%, Power wiring resistan

2024-02-22 09:00:24 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.


'There are 4 incident tickets.'

In [39]:
graphchain.invoke({"question":"what did the study concluded"}).get("result")



> Entering new GraphCypherQAChain chain...


2024-02-22 09:00:25 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


Generated Cypher:
MATCH (s:Entity)-[:CONCLUDES]->(c:Entity)
RETURN c
Full Context:
[{'c': {'id': 'only "about 13% of methane emissions are covered"'}}, {'c': {'id': 'only "about 13% of methane emissions are covered by methane mitigation policies"'}}, {'c': {'id': 'effectiveness of methane mitigation policies "is far from clear"'}}]


2024-02-22 09:00:25 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.


'The study concluded that only about 13% of methane emissions are covered by methane mitigation policies. Additionally, the effectiveness of these policies is far from clear.'

## Langchain tools

To combine vector retriever and Knowledge Graphs we can use langchain tools and an agent executor which combines it

In [40]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, create_structured_chat_agent

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser


In [41]:
#adding vector retriever as tool



toolslgc = [
    
    Tool(
        name="copilot_retriever",
        func=lambda q:vectorchain.invoke({"question":q}).get("answer"),
        description="useful for when you want to answer questions about RCA documents and tickets using FAISS.",
        return_direct=False,
    ),
    Tool(
        name="Copilot_Knowledge",
        func=lambda q:graphchain.invoke({"question":q}).get("result"),
        description="useful for when you want to answer questions about neo4j kg using langchain.",
        return_direct=False,
    ),

]

llm_with_tools= llmrca.bind_tools(toolslgc)

In [42]:
print(toolslgc[0].run("what is the first incident ticket about?"))
print(toolslgc[1].run("what did the study concluded?"))



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='where and why did the production stoppage occurred'), AIMessage(content='Based on the provided incident tickets, the production stoppage occurred on Assembly Line 3. The root cause of the stoppage can be attributed to a sudden and unplanned event. To determine the exact reason for the stoppage, a detailed analysis using Root Cause Analysis (RCA) and Failure Mode and Effects Analysis (FMEA) would be required. However, some potential factors that could have contributed to the stoppage include issues with motor performance, such as high motor surface temperature and load on the motor, as well as potential voltage stability and power wiring resistance issues. Additionally, factors such as dust accumulation and lubrication frequency could also impact 

2024-02-22 09:00:26 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-22 09:00:26 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/azure_embedding/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                ﻿Incident Ticket ID: INC0007654
Issue Description: Production Line 2 experienced a sudden disruption during the morning shift due to unexpected shutdown of Benchmill impacting the production schedule.
Parameter Values: Airflow RAirflow Rate : 70 CFM, Ambient Temperature : 35 degree celsius, Motor surface temperature : 80 degree celsius, Motor rated power : 1.5 HP,
Load on motor :

2024-02-22 09:00:26 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
The first incident ticket (INC0007654) describes a sudden disruption in Production Line 2 during the morning shift due to the unexpected shutdown of the Benchmill.


> Entering new GraphCypherQAChain chain...


2024-02-22 09:00:27 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


Generated Cypher:
MATCH (:Entity)-[:CONCLUDES]->(conclusion:Entity)
RETURN conclusion
Full Context:
[{'conclusion': {'id': 'only "about 13% of methane emissions are covered"'}}, {'conclusion': {'id': 'only "about 13% of methane emissions are covered by methane mitigation policies"'}}, {'conclusion': {'id': 'effectiveness of methane mitigation policies "is far from clear"'}}]


2024-02-22 09:00:27 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.
The study concluded that only about 13% of methane emissions are covered by methane mitigation policies. Additionally, the effectiveness of these policies is far from clear.


In [43]:
!pip install langchainhub

Error processing line 1 of C:\Users\o_hol\AppData\Roaming\Python\Python311\site-packages\matplotlib-3.6.3-py3.11-nspkg.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 186, in addpackage
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 570, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [47]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llmazure, toolslgc, prompt)
agent_executor = AgentExecutor(agent=agent, tools=toolslgc, verbose=True)


In [48]:
querytemplate= "try using the functions Copilot_Knowledge first to answer the users query then the copilot retriever, this is the query:{question}" 

In [49]:
#THIS ANSWER SHOULD COME FROM THE GRAPH
agent_executor.invoke({"input": 
                       querytemplate.format(question="tell me everything you know about")}).get("output")





> Entering new AgentExecutor chain...


2024-02-22 09:00:56 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"



Invoking: `Copilot_Knowledge` with `tell me everything you know about`




> Entering new GraphCypherQAChain chain...


2024-02-22 09:00:58 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


Generated Cypher:
MATCH (e1:Entity)-[r]->(e2:Entity)
RETURN e1, r, e2
Full Context:
[{'e1': {'id': 'pair'}, 'r': ({'id': 'pair'}, 'CHALLENGE', {'id': 'theories of planetary formation'}), 'e2': {'id': 'theories of planetary formation'}}, {'e1': {'id': 'pair'}, 'r': ({'id': 'pair'}, 'BE', {'id': 'lowest-mass star hosting'}), 'e2': {'id': 'lowest-mass star hosting'}}, {'e1': {'id': 'Five employees'}, 'r': ({'id': 'Five employees'}, 'PUBLISH', {'id': 'tropical cyclone report'}), 'e2': {'id': 'tropical cyclone report'}}, {'e1': {'id': 'August'}, 'r': ({'id': 'August'}, 'REACH', {'id': 'new record high temperature'}), 'e2': {'id': 'new record high temperature'}}, {'e1': {'id': "world's oceans"}, 'r': ({'id': "world's oceans"}, 'REACH', {'id': 'new record high temperature'}), 'e2': {'id': 'new record high temperature'}}, {'e1': {'id': 'Hurricane Ian'}, 'r': ({'id': 'Hurricane Ian'}, 'UPGRADES', {'id': 'Category 4 to Category 5'}), 'e2': {'id': 'Category 4 to Category 5'}}, {'e1': {'id': 'Hurr

...

2024-02-22 09:01:01 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/azure_embedding/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                RCA Reference Guide
Please refer the systematic sequence of steps listed below to methodically identify and address root
cause issues contributing to various issues in production quality.
1)Check the cooling mechanisms: Standard Airflow rate through cooling vents is >50 CFM.
2)Evaluate ambient temperature around the motor: Standard Value is 20-25 degrees Celsius.
3)Motor surface temperature during 

2024-02-22 09:01:02 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-22 09:01:03 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
As a Manufacturing AI Assistant, my expertise lies in the field of manufacturing processes, quality control, root cause analysis, and failure mode and effects analysis (FMEA). I can provide information and insights on various topics related to these areas, including production line optimization, quality improvement strategies, troubleshooting manufacturing issues, identifying root causes of problems, analyzing failure modes, and implementing preventive measures.Based on the information available, it seems that the Copilot_Knowledge function provided some general information about various topics, but it did not provide specific details about a particular subject. On the other hand, the copilot_retriever function, which is specialized in manufacturing processes, quality control, and root cause analysis, provided more relevant information related to its expertise.

If you have a specific topic or question in mind, please let me know, and I'll do my be

"Based on the information available, it seems that the Copilot_Knowledge function provided some general information about various topics, but it did not provide specific details about a particular subject. On the other hand, the copilot_retriever function, which is specialized in manufacturing processes, quality control, and root cause analysis, provided more relevant information related to its expertise.\n\nIf you have a specific topic or question in mind, please let me know, and I'll do my best to assist you further."

In [17]:
#THIS ANSWER SHOULD COME FROM THE VECTOR
agent_executor.invoke({"input": 
                       querytemplate.format(question="Give me an RCA on ticket 1")}).get("output")





> Entering new AgentExecutor chain...


2024-02-21 21:37:59 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 404 model_error"


NotFoundError: Error code: 404 - {'error': {'message': 'Unrecognized request argument supplied: tools', 'type': 'invalid_request_error', 'param': None, 'code': None}}

### LLama index graph test

In [142]:
from dataclasses import dataclass
@dataclass 
class RAG2:
    memory_conversation  = None
    def __init__(self,llm,graph,type="langchain",embeddings=None,usememory=False):
        self.llm = llm
        self.graph = graph
        self.type = type
        self.embeddings = embeddings
        self.usememory = usememory

    @staticmethod
    def memory(type='langchain',memory_key="chat_history",output_key='result',rmessages=True,readmem=None):
        if readmem:
            if type.lower()=="langchain":
                from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
                memory = ConversationBufferMemory(
                memory_key=memory_key, return_messages=rmessages)
                #output_key=output_key)
                
                readonlymemory = ReadOnlySharedMemory(memory=memory)

                return memory
        else: 
            return None
    

    def vector_retriever(self,store="chroma",**kwargs):
        texts = kwargs.get("texts")
        if store.lower()=="chroma":
            if not self.embeddings:
                self.embeddings = getembeddings(model=kwargs.get("model"),dep_name = kwargs.get("dep_name"),api_key = kwargs.get("api_key"),api_base= kwargs.get("api_base"))
            

            from langchain.chains import RetrievalQA
            persist_directory = kwargs.get("persist_directory","filesdb")
            search_kwargs = kwargs.get("search_kwargs",{"k": 3})
            from langchain.vectorstores import Chroma
            
            db = Chroma.from_documents(texts,self.embeddings,persist_directory= persist_directory)
            vec_retriever = db.as_retriever(search_kwargs=search_kwargs)
         
            return vec_retriever  

            
        elif store.lower()=="faiss":
            #to be implemented
            from langchain_community.vectorstores import FAISS
            from langchain_core.output_parsers import StrOutputParser
            from langchain_core.prompts import ChatPromptTemplate
            from langchain_core.runnables import RunnablePassthrough
            from langchain_openai import ChatOpenAI, OpenAIEmbeddings
            if not self.embeddings:
                self.embeddings = getembeddings(model=kwargs.get("model"),dep_name = kwargs.get("dep_name"),api_key = kwargs.get("api_key"),api_base= kwargs.get("api_base"))
            
            if kwargs.get("text_embeddings"):
                text_embeddings =kwargs.get("text_embeddings")
                vectorstore = FAISS.from_embeddings(text_embeddings=list(zip(text_embeddings[0],text_embeddings[1])), embedding = self.embeddings)
                retriever = vectorstore.as_retriever()
                return retriever
            
            else:
            
                vectorstore = FAISS.from_texts(texts, embedding=OpenAIEmbeddings())
                retriever = vectorstore.as_retriever()
                template = """Answer the question based only on the following context:{context} 
                Question: {query}"""
                prompt = ChatPromptTemplate.from_template(template)
                retrieval_chain = ({"context": retriever, "query": RunnablePassthrough()} | prompt
                                    | self.llm| StrOutputParser())
                return retrieval_chain
        
    def build_chain(self,**kwargs):
        #memory = self.memory() if (readmem := self.memory()) else None
        method = kwargs.get("method","graph")
        
        #PROMPT TEMPLATE        
        from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate,PromptTemplate

        
        #####

        memory = self.usememory
        verbose = kwargs.get("verbose",True)

        if memory:
            prompt = PromptTemplate(input_variables=["chat_history", "question"], template="{chat_history} {question}")
        else:
            prompt = PromptTemplate(input_variables=["question"],template ="{question}")

        self.memory_conversation = self.memory(readmem=memory) if not self.memory_conversation else self.memory_conversation

        if kwargs.get("falkor"):
            from langchain.chains import FalkorDBQAChain
            chain = FalkorDBQAChain.from_llm(llm=self.llm, graph=self.graph, verbose=verbose, memory=self.memory_conversation, **kwargs)
        else:
            if method.lower()=="graph":
                #general chain for graphs
                print("building graph chain")
                from langchain.chains import GraphCypherQAChain
                #Prompt template
                        
                general_system_template = r""" 
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                {context}
                ----
                """
                general_user_template = "Question:```{question}```"
                messages = [
                            SystemMessagePromptTemplate.from_template(general_system_template),
                            HumanMessagePromptTemplate.from_template(general_user_template)
                ]
                qa_prompt = ChatPromptTemplate.from_messages( messages )

                chain = GraphCypherQAChain.from_llm(llm=self.llm, prompt=prompt,graph=self.graph,validate_cypher = True, 
                                                    verbose=verbose,memory=self.memory_conversation, promptTemplate=qa_prompt,**kwargs)
                chain.input_key = "question"

            elif method.lower()=="vector":
                print("Building vector chain ")
                retriever = kwargs.get("vec_retriever")
                from langchain.chains import RetrievalQA
                from langchain.chains import ConversationalRetrievalChain
                
                # template = ("Combine the chat history and follow up question into "
                # "a standalone question. Chat History: {chat_history}"
                # "Follow up question: {query} ")
                template = ("Every time you respond you should start with This is Copilot your AI assistant")
                prompt = PromptTemplate.from_template(template)
                

                #chain = RetrievalQA.from_chain_type(llm=self.llm, chain_type="stuff", retriever=retriever,return_source_documents=True,verbose=verbose)
                        
                general_system_template = r""" 
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                {context}
                ----
                """
                general_user_template = "Question:```{question}```"
                messages = [
                            SystemMessagePromptTemplate.from_template(general_system_template),
                            HumanMessagePromptTemplate.from_template(general_user_template)
                ]
                qa_prompt = ChatPromptTemplate.from_messages( messages )
                
                chain = ConversationalRetrievalChain.from_llm(self.llm,retriever,memory=self.memory_conversation,get_chat_history=lambda h :h,
                                                              verbose=verbose,
                                                              combine_docs_chain_kwargs={"prompt": qa_prompt}) 
                
                #chain = RetrievalQA.from_chain_type(llm=self.llm, chain_type="stuff", retriever=retriever,
                #                                    return_source_documents=True,verbose=verbose,memory=self.memory_conversation,prompt=prompt)

        return chain


    def get_retriever(self,**kwargs):
        method = kwargs.get("method","graph")
        complexity = kwargs.get("complexity","simple")
        
        if method.lower()=='graph':
            if self.type.lower()=="langchain":
                from langchain.chains import GraphCypherQAChain
                if complexity.lower()=='simple':
                    print("SIMPLE RAG LANGCHAIN")
                    if usememory := kwargs.get("usememory") or self.memory_conversation:
                        print("With Read memory")
                    
                    chain = self.build_chain(**kwargs)
                    return chain
                
                    
            elif self.type.lower()=="llama":
                if complexity.lower()=='simple':
                    nl2graph = kwargs.get("nl2graph",False)
                    print(nl2graph)
                    from llama_index.query_engine import RetrieverQueryEngine
                    from llama_index.retrievers import KnowledgeGraphRAGRetriever                    
                    graph_rag_retriever = KnowledgeGraphRAGRetriever(        
                    storage_context=kwargs.get("storage_context"),
                    service_context=kwargs.get("service_context"),
                    llm=self.llm,
                    verbose=True,
                    with_nl2graphquery=nl2graph)
                    query_engine = RetrieverQueryEngine.from_args(graph_rag_retriever, service_context=kwargs.get("service_context"))
                    return query_engine

                
                elif complexity.lower()=="custom":
                    if self.type.lower()=="llama":
                        from llama_index import get_response_synthesizer
                        from llama_index.query_engine import RetrieverQueryEngine
                        # create custom retriever
                        vector_retriever = VectorIndexRetriever(index=vector_index)
                        kg_retriever = KGTableRetriever(index=kg_index, retriever_mode="keyword", include_text=False)
                        custom_retriever = CustomRetriever(vector_retriever, kg_retriever)
                        # create response synthesizer
                        response_synthesizer = get_response_synthesizer(service_context=service_context,response_mode="tree_summarize")
                        custom_query_engine = RetrieverQueryEngine(retriever=custom_retriever,response_synthesizer=response_synthesizer)
                        return custom_query_engine
                    elif self.type.lower()=='langchain':
                        from langchain.agents import initialize_agent, Tool

                        simple = self.get_retriever("simple",**kwargs)
                        
                        tools = [
                            Tool(name= "GraphRAG",
                                 func= simple.run,
                                 description= "Simple RAG from knowledge graph"),
                            #add more RAG tools
                        ]
        elif method.lower()=="vector":
            print("Getting vector retriever")
            vec_retriever = self.vector_retriever(**kwargs)

            chain = self.build_chain(vec_retriever=vec_retriever,**kwargs)
            return chain



    def response(self,query,complexity="simple",**kwargs):
        method = kwargs.get("method","vector")
        verbose = kwargs.get("verbose",True)
        retriever = self.get_retriever(**kwargs)
        try:
            if method.lower()=="vector":
                print("I AM USING VECTOR LETS GO")
                result =  retriever({"question": query}).get('answer')
                
            elif method.lower()=="graph":
                if self.type.lower()=='langchain':
                    print('using retriever({"query": query}).get("answer")')
                    #result = retriever({"question": query}).get('answer')#
                    result = retriever.invoke({"question": query}).get('result') 
                    pickled_str = pickle.dumps(self.memory_conversation)
                elif self.type.lower()=='llama':
                    result = retriever.query(query)
                    if 'Empty Response' in result.response:
                        raise("Error empty result or no information found")
                    else:
                        return result.response
            
            if not result or "I'm sorry, but I don't have" in result:
                print('Not using RAG')
                raise ("Error empty result or no information found in the KG")
            elif "I'm sorry, but I don't have" in result:
                raise("ERROR 404")
            else:
                return result

        except Exception as e:
            print(e)
            if self.type.lower()=='langchain':
                from langchain.schema import HumanMessage
                #message = HumanMessage(content=query)
                from langchain.chains import ConversationChain
                from langchain.prompts.prompt import PromptTemplate
                if self.usememory:
                    print("Normal LLM response with same memory")
                    prompt = PromptTemplate(input_variables=["chat_history", "question"], template="{chat_history} {query}")
                    chat = ConversationChain(llm=self.llm,verbose=kwargs.get("verbose",True),prompt=prompt,memory=self.memory_conversation ,input_key="question")
                    return chat.invoke({"question":query}).get("response")
                else:
                    from langchain.schema import HumanMessage
                    message = HumanMessage(content=query)
                    return self.llm.invoke([message])

            else:
                #return response with llama index self.llm(response)
                pass
            

In [21]:
def getllm(openai=True, azure=False, mode="langchain",**kwargs):
    print("Note: to use the LLM use it as msg = HumanMessage(content= your query) \n llm(messages = [msg]) ")
    if openai:
        temperature = kwargs.get("temperature",0)
        if azure:
            print("Using Azure OpenAI")
            # from langchain.chat_models import AzureChatOpenAI
            from langchain_openai import AzureChatOpenAI
            auth = kwargs.get("auth")
            if auth.lower()=="token":
                if mode.lower()=="langchain":
                    params = {"model": kwargs.get("model"),
                        "azure_deployment": kwargs.get("dep_name"),
                        "openai_api_version": kwargs.get("api_version"),
                        "temperature": temperature,
                        "openai_api_key": kwargs.get("api_key"),
                        "azure_endpoint": kwargs.get("api_base"),
                        "max_tokens":kwargs.get("max_tokens",256)}
                    llm = AzureChatOpenAI(**params)
                elif mode.lower()=="llama":
                    print("Llama's version")
                    #from llama_index.llms import AzureOpenAI
                    from llama_index.llms.azure_openai import AzureOpenAI
                    import openai
                    openai.api_type = "azure"
                    # openai.api_base =  kwargs.get("api_base") #"INSERT AZURE API BASE"
                    # openai.api_version = kwargs.get("api_version","2023-05-15")
                    
                    llm = AzureOpenAI(engine= kwargs.get("dep_name"),temperature=0,model=kwargs.get("model"),
                                     azure_deployment =kwargs.get("dep_name"),azure_endpoint = kwargs.get("api_base"),
                                     api_version = kwargs.get("api_version","2023-05-15"),api_key= kwargs.get("api_key"))
                return llm
            elif auth.lower() == "azure_ad":
                #from azure.identity import DefaultAzureCredential
                #credential = DefaultAzureCredential()
                #Set the API type to `azure_ad`
                #os.environ["OPENAI_API_TYPE"] = "azure_ad"
               ##Set the API_KEY to the token from the Azure credential
                #os.environ["OPENAI_API_KEY"] = credential.get_token("https://cognitiveservices.azure.com/.default").token            elif auth.lower()=="azure_ad_provider":
                llm = "Not implemented"
            elif auth == "azure ad_provider":
                #managed_identity:
                #from azure.identity import ChainedTokenCredential, ManagedIdentityCredential, AzureCliCredential   
                #credential = ChainedTokenCredential(
                #ManagedIdentityCredential(),
                #AzureCliCredential())
                llm = "not implemented"
        else:
            from langchain_openai import OpenAI
            api_base = kwargs.get("api_base_url")
            api_key =kwargs.get("api_key","NULL")
            llm = OpenAI(
                         base_url = api_base,
                         openai_api_key = api_key,**kwargs)
            return llm 
    else: #huggingface model
        pass    



def llm_cost(llm,message) -> "Total cost in USD":
    from langchain.callbacks import get_openai_callback
    with get_openai_callback() as cb:
        llm([message])
        cost=format(cb.total_cost, '.6f')
        print(f"Total Cost (USD): ${cost}")
    return cost



    def get_embeddings(self,documents=None,lib='langchain'):
#         if not documents:
#             documents = pass
        #huggingface or langchain
        if lib=='langchain':
            from langchain.embeddings import OpenAIEmbeddings
            embeddings_model = OpenAIEmbeddings()
            embeddings = embeddings_model.embed_documents(documents)
            return emebeddings
        else:
            from langchain.embeddings import HuggingFaceEmbeddings
            embeddings = HuggingFaceEmbeddings(model_name=self.huggingembed)
            return embeddings

In [22]:

def get_graphdb(args,**kwargs):
    graphdb = kwargs.get("graphdb")
    url,username,password,database = kwargs.get("url"), kwargs.get("username"), kwargs.get("password"), kwargs.get("database")
    if args.lower()=='llama':
        #from llama_index.storage.storage_context import StorageContext
        from llama_index.core import StorageContext
        print('Using LLama Index graph class')
        if graphdb.lower()=='neo4j' or 'neo4j' in url:
            from llama_index.graph_stores.neo4j import Neo4jGraphStore

            graph_store = Neo4jGraphStore(
                username=username,
                password=password,
                url=url,
                database=database,
            )
            #build storage context
            storage_context = StorageContext.from_defaults(graph_store=graph_store)
            return graph_store, storage_context
        elif graphdb.lower()=='nebula':
            from llama_index.graph_stores.nebula import NebulaGraphStore
            #TBD get varibales from kwargs
            graph_store = NebulaGraphStore(
            space_name=kwargs.get("space_name"),
            edge_types=kwargs.get("edge_types"),
            rel_prop_names=kwargs.get("rel_prop_names"),
            tags=kwargs.get("tags"))
            storage_context = StorageContext.from_defaults(graph_store=graph_store)
            return graph_store, storage_context
        
        else:
            raise("Error not implemented for that graph database")
    

    elif args.lower()=='langchain':
        print('Using langchain graph class')
        if graphdb.lower()=='neo4j' or 'neo4j' in url:
            from langchain.graphs import Neo4jGraph
            graph= Neo4jGraph(
               url=url,
               username=username,
               password=password
            )
    
            return graph
        elif graphdb.lower()=="falkordb":
            print("Using falkordb")
            from langchain.graphs import FalkorDBGraph
            graph = FalkorDBGraph(database = database,host=url,port=6379, username=username,password = pwd)


        else:
            raise("Error not implemented for that graph database")


In [18]:
#loading 
llmazure =getllm(openai=True,azure =True,mode = "langchain", auth="token",model="gpt-35-turbo-16k",dep_name = "chat-bot-trail",            
                  api_version= "2023-12-01-preview", api_key = "1bbc7a5ceb0e4677888f22d1eb4c8617",api_base = "https://chat-bot-iot.openai.azure.com/")#([message])
#LLAMA'S VERSION
llmazure2 =getllm(openai=True,azure =True,mode = "llama", auth="token",model="gpt-35-turbo-16k",dep_name = "chat-bot-trail",            
                  api_version= "2023-05-15", api_key = "1bbc7a5ceb0e4677888f22d1eb4c8617",api_base = "https://chat-bot-iot.openai.azure.com/")#([message])

Note: to use the LLM use it as msg = HumanMessage(content= your query) 
 llm(messages = [msg]) 
Using Azure OpenAI
Note: to use the LLM use it as msg = HumanMessage(content= your query) 
 llm(messages = [msg]) 
Using Azure OpenAI
Llama's version


C:\Users\o_hol\AppData\Roaming\Python\Python311\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
#INITIATE THE GRAPH STORE
url = "neo4j+s://b75e0bd8.databases.neo4j.io"
username= "neo4j"
password = "aziMmumvuVru-rgeKuO0zT7mEtTxu29Jgrmp6lFKv0w"
graph_store = get_graphdb("langchain",graphdb="neo4j",url=url,username=username,password=password)
#GRAPH STORE LLAMA 
graph_store_llama, storage_context = get_graphdb("llama",graphdb="neo4j",url=url,username=username,password=password)

Using langchain graph class
Using LLama Index graph class


ImportError: cannot import name 'Neo4jGraphStore' from 'llama_index.graph_stores' (unknown location)

In [ ]:
from llama_index.core.query_engine import KnowledgeGraphQueryEngine
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

In [ ]:
rag2 = RAG2(llmazure2,graph_store_llama,type="llama")
service_context= ""


In [ ]:
graph_rag_retriever = KnowledgeGraphRAGRetriever(        
                    storage_context=storage_context,
                    llm=llmazure2,
                    verbose=True,
                    with_nl2graphquery=True)

#query_engine = RetrieverQueryEngine.from_args(graph_rag_retriever, service_context=kwargs.get("service_context"))

In [28]:
graph_rag_retriever = KnowledgeGraphRAGRetriever(        
                    storage_context=storage_context,
                    llm=llmazure2,
                    verbose=True,
                    with_nl2graphquery=False)


In [143]:
llmazure2

AzureOpenAI(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001D8297BA550>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x000001D8B7719440>, completion_to_prompt=<function default_completion_to_prompt at 0x000001D8B779C360>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='gpt-35-turbo-16k', temperature=0.0, max_tokens=None, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='1bbc7a5ceb0e4677888f22d1eb4c8617', api_base='https://api.openai.com/v1', api_version='2023-05-15', engine='chat-bot-trail', azure_endpoint='https://chat-bot-iot.openai.azure.com/', azure_deployment='chat-bot-trail', use_azure_ad=False)

In [144]:
query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
    llm = llmazure2)

In [145]:
query_engine.query(
    "what did the study concluded",
).response

2024-02-21 14:20:47 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-21 14:20:47 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-21 14:20:47 [ERROR]: Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
2024-02-21 14:20:47 [ERROR]: Failed to write data to connection IPv4Address(('b75e0bd8.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
2024-02-21 14:20:48 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


'The study concluded that a moderate decrease in body temperature extends lifespan.'

In [146]:
graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    with_nl2graphquery=True,
    llm = llmazure2
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
    llm = llmazure2
)

In [147]:
query_engine_with_nl2graphquery.query(
    "what did the study concluded",
).response

2024-02-21 14:21:04 [WARNING]: Error in retrieving from nl2graphquery: 'NoneType' object has no attribute 'kwargs'
2024-02-21 14:21:05 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-21 14:21:06 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-21 14:21:06 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


'The study concluded that a moderate decrease in body temperature extends lifespan.'

In [148]:
#add it to langchain tools
from llama_index.core.langchain_helpers.agents import (
    IndexToolConfig,
    LlamaIndexTool,
)

tool_config = IndexToolConfig(
    query_engine=query_engine_with_nl2graphquery,
    name=f"KG Index",
    description=f"useful for when you want to answer queries about neo4j",
    tool_kwargs={"return_direct": False},
)

tool = LlamaIndexTool.from_tool_config(tool_config)




In [149]:
tool.run("tell me all you know about a study")




2024-02-21 14:21:08 [WARNING]: Error in retrieving from nl2graphquery: 'NoneType' object has no attribute 'kwargs'
2024-02-21 14:21:08 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-21 14:21:09 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2024-02-21 14:21:10 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


'The study is related to various subjects such as a moderate decrease in body temperature extending lifespan, neuro-molecular mechanics, disappearance, more than 50% of freshwater lakes, population declines, school curricula including certain topics, carbon taxation approaches, release of methane from Arctic glacial retreat, and the assembly of 43 diverse Y chromosomes.'

## Other test

In [62]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, create_structured_chat_agent

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser


In [76]:
#llmazure.bind_tools(tool)
tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: query_engine_with_nl2graphquery.query(q),
        description="useful for when you want to answer questions about neo4j kg.",
        return_direct=False,
    ),
]

llm_with_tools= llmazure.bind_tools(tools)

import openai


In [151]:
tools[0].run("what did the study concluded?")

NameError: name 'tools' is not defined

In [152]:
from langchain.agents.agent_toolkits.conversational_retrieval.openai_functions import create_conversational_retrieval_agent

agent_executor = create_conversational_retrieval_agent(llm=llmazure, tools=[
    tools[0]], verbose=True)
agent_executor({"input": "what did the study concluded about the lifespan?"})

NameError: name 'tools' is not defined

In [78]:
llm_with_tools.invoke("what did the study concluded?").content

2024-02-19 20:35:00 [INFO]: HTTP Request: POST https://chat-bot-iot.openai.azure.com//openai/deployments/chat-bot-trail/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"


"I'm sorry, but I don't have enough information to answer your question. Could you please provide more context or specify which study you are referring to?"

In [43]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful manufacturing AI Assistant, you specialize in Quality department but have expertise in general manufacturing",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
    
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [44]:
agent_executor("hi")

ValueError: A single string input was passed in, but this chain expects multiple inputs (set()). When a chain expects multiple inputs, please call it by passing in a dictionary, eg `chain({'foo': 1, 'bar': 2})`